In [1]:
library(bigrquery)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Store the project ID
projectid = "yhcr-prd-phm-bia-core"

In [6]:
# Diabetes
sql1 <- "SELECT *  FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.Diabetes`"
tb1 <- bq_project_query(projectid, sql1)
diabetes <-bq_table_download(tb1)
diab <- diabetes %>% select(person_id, any_diabetes)

In [4]:
# Secondary Care Episodes table
sql2 <- "SELECT person_id, episode_start_date, episode_end_date FROM `yhcr-prd-phm-bia-core.CB_FDM_Warehouse_V2.tbl_episode` 
WHERE episode_start_date BETWEEN '2002-01-01' AND '2022-12-31'"
tb2 <- bq_project_query(projectid, sql2)
episodes <-bq_table_download(tb2)

In [13]:
# Join diabetes and episodes tables
diab_epi <- left_join(diab, episodes, by = "person_id")
diab_epi <- diab_epi %>% filter(!is.na(episode_start_date))
head(diab_epi)

person_id,any_diabetes,episode_start_date,episode_end_date
<int>,<int>,<chr>,<chr>
12975170,1,2013-03-05,2013-03-23
12975170,1,2020-04-29,2020-04-30
12975170,1,2018-02-21,2018-02-21
12975170,1,2013-03-30,2013-04-21
12975170,1,2013-05-23,2013-05-29
12975170,1,2018-08-13,2018-08-13


In [16]:
# Calculate length of stay
diab_epi <- diab_epi %>%
    mutate(los_all_days = round(difftime(episode_end_date, episode_start_date, units = "days"))) %>%
    mutate(los_all_days = as.integer(los_all_days))

In [ ]:
# Keep only one row per patient? Based on?

In [23]:
table(diab_epi$los_all_days == 1)

dep <- diab_epi %>% 
    filter(diab_epi$los_all_days >= 2)

summary(dep$los_all_days)
IQR(dep$los_all_days)


 FALSE   TRUE 
162732  35641 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.000   2.000   4.000   7.232   8.000 379.000 

[1] 6